# LangChain x Pinecone

https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/

In [ ]:
import os
import time
from pinecone import Pinecone, ServerlessSpec

# indexの初期化

In [ ]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
pc

In [ ]:
index_name = "langchain-test-index"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

In [ ]:
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
existing_indexes

In [ ]:
pc.describe_index(index_name)

In [ ]:
index = pc.Index(index_name)
index

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# Vectorstoreに文書を追加

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

# クエリ

In [ ]:
vector_store

In [ ]:
results = vector_store.similarity_search(
    query="LangChain provides abstractions to make working with LLMs easy",
    k=5,
    filter={"source": "tweet"}
)
results

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
    k=1,
    filter={"source": "news"}
)
results

In [ ]:
vector_store.as_retriever()